In [1]:
HOME_DIR = "/home_remote"
HOME = "/home/thi.tra.my.nguyen"

import pandas as pd
import glob
import re
import nltk
from nltk.tokenize import RegexpTokenizer
import zipfile
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import fasttext
import fasttext.util


import ast
import os

### Word Embedding

Using Wiki pretrained model

In [2]:
ft = fasttext.load_model('/home_remote/fastText/cc.en.300.bin')

In [3]:

# Function to get word embeddings for a document
def get_document_embeddings(document, max_length=100):
    # Tokenize the document and get the first max_length word vectors
    tokens = document.split()[:max_length]
    
    # Apply zero-padding if the document is shorter than max_length
    if len(tokens) < max_length:
        padding_count = max_length - len(tokens)
        tokens.extend(['<PAD>'] * padding_count)

    # Get word embeddings for each token
    embeddings = [ft.get_word_vector(token) for token in tokens]

    return np.array(embeddings)


### Convolutional Neural Network

In [4]:
positives_df_path = os.path.join(HOME_DIR, "positive_df.pkl")
negatives_df_path = os.path.join(HOME_DIR, "negative_df.pkl")

positives = pd.read_pickle(positives_df_path)
negatives = pd.read_pickle(negatives_df_path)

positives['Label'] = 1
negatives['Label'] = 0

#concatenate the two dataframes
df = pd.concat([positives, negatives], ignore_index=True)

In [5]:
subject = df.TrainSubjectId.unique()
#split the subjects into train and test
train_subject, test_subject = train_test_split(subject, test_size=0.2, random_state=42, shuffle=True)
df_train = df[df.TrainSubjectId.isin(train_subject)]
df_test = df[df.TrainSubjectId.isin(test_subject)]
df_train = df_train.sample(frac=1, random_state=42)
df_test = df_test.sample(frac=1, random_state=42)
y_train = df_train.Label
y_test = df_test.Label
X_train = df_train['Text']
X_test = df_test['Text']

In [6]:
# Custom CReLU activation function
def crelu(x):
    pos = K.relu(x)
    neg = K.relu(-x)
    return K.concatenate([pos, neg], axis=-1)

In [7]:
# Word embedding dimensionality
word_vector_dimensions = 300

# Model
model = Sequential()

# Convolutional Layer
model.add(Conv1D(filters=100, kernel_size=2, input_shape=(100, 300)))
model.add(Activation(crelu))
model.add(Activation(lambda x: x * -1))  # Negated activation for CReLU

# 1-Max Pooling Layer
model.add(MaxPooling1D(pool_size=99))

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(200))

# Dropout Layer
model.add(Dropout(0.4))

# 3 Fully Connected Layers
model.add(Dense(100, activation=crelu))
model.add(Dense(50, activation=crelu))

# Final Layer
model.add(Dense(2, activation='softmax'))

# Display the model summary
model.summary()

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['f1'])


2023-12-04 21:08:25.140207: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 21:08:25.682408: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10794 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0000:8d:00.0, compute capability: 3.7


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 99, 100)           60100     
                                                                 
 activation (Activation)     (None, 99, 200)           0         
                                                                 
 activation_1 (Activation)   (None, 99, 200)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1, 200)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 200)               40200     
                                                        

In [9]:
#convert X_train and X_test to word embedding using get_document_embeddings function
X_train = np.array([get_document_embeddings(document) for document in X_train])
X_test = np.array([get_document_embeddings(document) for document in X_test])

In [12]:

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Get the 98th percentile of the predictions
percentile_98 = np.percentile(y_pred, 98)

print("98th Percentile of Predictions:", percentile_98)


2023-12-04 21:44:29.894843: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 28491240000 exceeds 10% of free system memory.
2023-12-04 21:44:58.498608: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 26.53GiB (rounded to 28491240192)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-12-04 21:44:58.498683: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2023-12-04 21:44:58.498721: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 19, Chunks in use: 19. 4.8KiB allocated for chunks. 4.8KiB in use in bin. 292B client-requested in use in bin.
2023-12-04 21:44:58.498744: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 3

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

203be0700 of size 256 next 4
2023-12-04 21:44:58.499309: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 4203be0800 of size 256 next 5
2023-12-04 21:44:58.499327: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 4203be0900 of size 512 next 6
2023-12-04 21:44:58.499343: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 4203be0b00 of size 256 next 9
2023-12-04 21:44:58.499362: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 4203be0c00 of size 256 next 10
2023-12-04 21:44:58.499379: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 4203be0d00 of size 1024 next 11
2023-12-04 21:44:58.499399: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 4203be1100 of size 256 next 12
2023-12-04 21:44:58.499415: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 4203be1200 of size 256 next 14
2023-12-04 21:44:58.499433: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 4203be1300 o

In [14]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14367927769888350324
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11319115776
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 323540787456691409
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:8d:00.0, compute capability: 3.7"
xla_global_id: 416903419
]


2023-12-04 21:47:40.188720: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 10794 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0000:8d:00.0, compute capability: 3.7
